In [ ]:
pd.set_option('mode.chained_assignment', None)

data = pd.read_csv('bonds_w_exp_returns.csv')
# Check if model is valid and call the corresponding optimization function
data = data.sort_values(by='Date')
data['Date'] = pd.to_datetime(data['Date'])

model = 'cVaR'
constraints = []
# Initial budget to invest ($100,000)
initialVal = 100000  

# Length of investment period 
investPeriod = 1

# Identify the tickers and the dates
tickers = data['SecurityId'].unique()
dates = data['Date'].unique()

n = len(tickers)   # Number of bonds
NoPeriods = len(dates) // investPeriod  

# Preallocate space for portfolio weights (x0 will track turnover)
x = np.zeros([n, NoPeriods])
x0 = np.zeros([n, NoPeriods])

# Preallocate space for portfolio value, turnover, and transaction costs
currentVal = np.zeros([NoPeriods + 1, 1])
currentVal[0] = initialVal
portfolio_returns = np.zeros(NoPeriods)
turnover = np.zeros([NoPeriods, 1])
transaction_costs = np.zeros([NoPeriods, 1])

lookback_window = 126  # Past 126 days
start_period = lookback_window // investPeriod  # Determine the first valid period

for period in range(start_period, NoPeriods):  # Start at period 126
    print("Testing period:", period)
    current_date = dates[period]  # The current day
    # Get the last 126 unique trading days (from the 'Date' column)
    end_date = current_date  # Current date is the last day
    start_date = dates[period - lookback_window]  # Start date is 126 days earlier

    # Filter data to only include the unique trading dates
    trading_days_in_range = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
    unique_trading_dates = trading_days_in_range['Date'].unique()

    if len(unique_trading_dates) < lookback_window:
        print(f"Skipping period {period}: not enough unique trading days.")
        continue
    
    # Now we use only the last 126 unique trading days
    # Sort the trading dates and slice the last 126 unique ones
    last_126_trading_days = unique_trading_dates[-lookback_window:]

    # Filter the original data to keep only rows from the last 126 unique trading days
    rolling_window_data = data[data['Date'].isin(last_126_trading_days)]

    # Pivot to create a matrix of daily returns (rows: days, columns: bonds)
    daily_returns_matrix = rolling_window_data.pivot(index='Date', columns='SecurityId', values='ExpectedReturn')

    # Drop any columns (bonds) with missing data
    daily_returns_matrix = daily_returns_matrix.dropna(axis=1)

    # Check if there are enough scenarios (days) and bonds
    if daily_returns_matrix.shape[0] < lookback_window or daily_returns_matrix.shape[1] < len(tickers):
        print(f"Skipping period {period}: insufficient historical data for bonds.")
        continue

    # Use the daily returns matrix as `scenario_returns`
    scenario_returns = daily_returns_matrix.values  # Convert to NumPy array
    weights = CVaR_optimization(scenario_returns, alpha=0.95)
    # Store weights
    if weights is None:
        print("model returned nothing")
    
    x[:, period] = weights
    
    current_bonds = data[data['Date'] == current_date]

    print(f"Weights: {weights}")
    
    # Portfolio calculations (as in your original code)
    portfolio_return = np.sum(weights * current_bonds['ExpectedReturn'])
    print("return: ", portfolio_return)
    currentVal[period + 1] = currentVal[period] + portfolio_return
    portfolio_returns[period] = portfolio_return

    # Turnover and transaction costs (as in your original code)
    turnover[period] = np.sum(np.abs(weights - x0[:, period])) / 2
    # Calculate transaction costs
    turnover_weights = np.abs(weights - x0[:, period])

    transaction_costs[period] = np.sum(turnover_weights * current_bonds['BidAskSpread'].values)
    currentVal[period + 1] -= transaction_costs[period]
    x0[:, period] = weights


excess_returns = portfolio_returns

# Calculate Sharpe ratio
SR = (gmean(excess_returns + 1) - 1) / excess_returns.std()

# Average turnover and cumulative transaction cost
avgTurnover = np.mean(turnover[1:])
total_transaction_cost = np.sum(transaction_costs)

print('Sharpe ratio: ', str(SR))
print('Avg. turnover: ', str(avgTurnover))
print('Total transaction costs: ', str(total_transaction_cost))